---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [42]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open('university_towns.txt') as file:
        raw_data = []
        for line in file:
            raw_data.append(line[:-1])
        #print(raw_data) #see the structure of the raw data, then the following lines will make sense
    
    Uni_town = []   
    for item in raw_data:
        if '[edit]' in item: #if '[edit]' is in this line of data, then this is the name of state. Have to remove '[edit]'
            State = item[:-6]
        elif '(' in item: #if there is '(' in this line of data, then it is a region name. Remove anything after (
            Town= item.split(" (")[0]
            Uni_town.append([State, Town])
        else: #it is a region name that is already clean
            Town = item
            Uni_town.append([State, Town])
            
    Uni_town_df = pd.DataFrame(Uni_town, columns = ['State', 'RegionName'])
    
    return Uni_town_df

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows = 7)
    df.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP'}, inplace=True)
    # only look at GDP data from the first quarter of 2000 onward
    cutoff_point = df[df['Quarter'] == '2000q1'].index[0]
    df = df[cutoff_point:].reset_index()
    df = df[['Quarter','GDP']]#keep the two important columns
    
    for i in range(0,len(df)-2):
        if df['GDP'].iloc[i] > df['GDP'].iloc[i+1] and df['GDP'].iloc[i+1] > df['GDP'].iloc[i+2]: #we are in recession
            recession_start_time = df['Quarter'].iloc[i]
            break
            
    
    return recession_start_time
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows = 7)
    df.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP'}, inplace=True)
    # only look at GDP data from the first quarter of 2000 onward
    cutoff_point = df[df['Quarter'] == '2000q1'].index[0]
    df = df[cutoff_point:].reset_index()
    df = df[['Quarter','GDP']]#keep the two important columns
    
    recession_start_time = get_recession_start()
    recession_start_index = df[df['Quarter'] == recession_start_time].index[0]
    for i in range(recession_start_index + 1,len(df)-2):
        if df['GDP'].iloc[i] < df['GDP'].iloc[i+1] and df['GDP'].iloc[i+1] < df['GDP'].iloc[i+2]: #recession has ended
            recession_end_time = df['Quarter'].iloc[i+2]
            break
       
    return recession_end_time

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows = 7)
    df.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP'}, inplace=True)
    # only look at GDP data from the first quarter of 2000 onward
    cutoff_point = df[df['Quarter'] == '2000q1'].index[0]
    df = df[cutoff_point:].reset_index()
    df = df[['Quarter','GDP']]#keep the two important columns
    
    #find start and end of the recession
    recession_start_time = get_recession_start()
    recession_start_index = df[df['Quarter'] == recession_start_time].index[0]
    recession_end_time = get_recession_end()
    recession_end_index = df[df['Quarter'] == recession_end_time].index[0]
    
    df_recession = df[recession_start_index:recession_end_index]
    recession_bottom_time = df_recession[df_recession['GDP'] == min(df_recession['GDP'])]['Quarter'].iloc[0]
    
    return recession_bottom_time

get_recession_bottom()

'2009q2'

In [40]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df_raw = pd.read_csv('City_Zhvi_AllHomes.csv')
    house_df = df_raw[['State', 'RegionName']]
    
    for i in range(2000, 2016):
        #quarter one of each year
        house_df[str(i)+'q1'] = df_raw[[str(i)+'-01' , str(i)+'-02', str(i)+'-03']].mean(axis = 1) 
        
        #quarter two of each year
        house_df[str(i)+'q2'] = df_raw[[str(i)+'-04', str(i)+'-05', str(i)+'-06']].mean(axis = 1)
        
        #quarter three of each year
        house_df[str(i)+'q3'] = df_raw[[str(i)+'-07', str(i)+'-08', str(i)+'-09']].mean(axis = 1)
        
        #quarter four of each year
        house_df[str(i)+'q4'] = df_raw[[str(i)+'-10', str(i)+'-11', str(i)+'-12']].mean(axis = 1)
        
    #2016 has the data up to its q4 (well q3 is partial...)
    house_df['2016q1'] = df_raw[['2016-01', '2016-02', '2016-03']].mean(axis = 1)
    house_df['2016q2'] = df_raw[['2016-04', '2016-05', '2016-06']].mean(axis = 1)
    house_df['2016q3'] = df_raw[['2016-07', '2016-08']].mean(axis = 1)

    house_df['State'] = house_df['State'].map(states)
    house_df.set_index(['State','RegionName'],inplace=True)
    
    return house_df

convert_housing_data_to_quarters().head()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [51]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start_q = get_recession_start()
    bottom_q = get_recession_bottom()
    ttest_data = convert_housing_data_to_quarters()
    ttest_data = ttest_data[[start_q, bottom_q]].reset_index()  #['State', 'RegionName'] are in index
    
    #calculate and add the price change ratio to the dataset
    ttest_data['price_change'] = ttest_data.apply(lambda x: (x[start_q]-x[bottom_q])/x[start_q], axis = 1)
    
    #combine the state and region names into one string to make comparison easier
    uni_towns = get_list_of_university_towns()
    uni_towns['combined_location'] = uni_towns.apply(lambda x: x['State']+' '+x['RegionName'], axis=1)
    ttest_data['combined_location'] = ttest_data.apply(lambda x: x['State']+' '+x['RegionName'], axis=1)
    ttest_data.drop(['State','RegionName',start_q, bottom_q],axis=1,inplace=1)
    #check if the region is a university town
    def is_it_uni_town(x):
        if x['combined_location'] in list(uni_towns['combined_location']):
            return 1
        else:
            return 0
    ttest_data['is_uni_town'] = ttest_data.apply(is_it_uni_town , axis = 1)

    #T-test
    uni_town_prices = ttest_data[ttest_data['is_uni_town'] == 1].loc[:,'price_change'].dropna()
    non_uni_town_prices = ttest_data[ttest_data['is_uni_town'] == 0].loc[:,'price_change'].dropna()
    
    p_value = list(ttest_ind(uni_town_prices, non_uni_town_prices))[1]
    #p_value = float("{0:.5f}".format(p_value)) #5 decimal points
    if p_value > 0.01:
        different = False
    else:
        different = True
        
    uni_price_mean = uni_town_prices.mean()
    non_uni_price_mean = non_uni_town_prices.mean()
    if uni_price_mean > non_uni_price_mean:
        better = "non-university town"
    else:
        better = "university town"
        
    result = (different, p_value, better)

    
    return result

run_ttest()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

(True, 0.0089755875407126172, 'university town')